In [ ]:
from functools import reduce
import sys
import onnx
from onnx.mapping import TENSOR_TYPE_TO_NP_TYPE
from collections import defaultdict

def inspect_onnx(model_path): 
    onnx_model = onnx.load(model_path)
    print(f"====== load model {model_path} done.")

    g = onnx_model.graph

    def process_node():
        op_counter = defaultdict(int)
        for node in g.node:
            name = node.name
            optype = name.split("/")[-1].split("_")[0]
            optype = node.op_type
            op_counter[optype] += 1

        print("*" * 50)
        for k, v in sorted(op_counter.items()):
            print(f"{k:40}:  {v}")
        print("*" * 50)
        print("Total OP num: ", reduce(lambda i, j: i + j, op_counter.values(), 0))

    def process_inout():
        print("\n", "*" * 50, "inputs:")
        for node in g.input:
            name = node.name
            dtype = str(TENSOR_TYPE_TO_NP_TYPE[node.type.tensor_type.elem_type])
            shape = [dim.dim_value for dim in node.type.tensor_type.shape.dim]
            print(f"{name:40}{dtype:<10}{shape}")
            
        print("\n", "*" * 50, "outputs:")
        for node in g.output:
            name = node.name
            dtype = str(TENSOR_TYPE_TO_NP_TYPE[node.type.tensor_type.elem_type])
            shape = [dim.dim_value for dim in node.type.tensor_type.shape.dim]
            print(f"{name:40}{dtype:<10}{shape}")

    def nodename_info():

        node_name_set = set(n.name for n in g.node) 
        valinfo_name_set = set(n.name for n in g.value_info)


        print(f"node  name len: {len(node_name_set)}") 
        print(f"vainf name len: {len(valinfo_name_set)}")

        a = node_name_set.update(valinfo_name_set)
        print(f"merge name len: {len(node_name_set)}")
    
    process_node()
    process_inout()
    nodename_info()

In [ ]:
import numpy as np
import onnxruntime as ort

def infer_onnx(model_path):
    # Load the ONNX model
    sess = ort.InferenceSession(model_path)
    onnx_to_numpy_dtype = {
        "tensor(float16)": np.float16,
        "tensor(float)": np.float32,
        "tensor(double)": np.float64,
        "tensor(int8)": np.int8,
        "tensor(int16)": np.int16,
        "tensor(int32)": np.int32,
        "tensor(int64)": np.int64,
        "tensor(uint8)": np.uint8,
        "tensor(uint16)": np.uint16,
        "tensor(uint32)": np.uint32,
        "tensor(uint64)": np.uint64,
        # 'tensor(bool)': np.bool,
    }

    feeds = {}
    for input in sess.get_inputs():
        # Get input name and shape
        input_name = input.name
        input_shape = input.shape
        numpy_type = onnx_to_numpy_dtype.get(input.type, None)
        # Generate random input
        input_data = np.random.random_sample(input_shape).astype(numpy_type)

        feeds[input_name] = input_data

    # Run the model
    result = sess.run(None, feeds)

    print([a.shape for a in result])
